In [1]:
## ouvrir le csv avec pandas
import pandas as pd
data_spam =pd.read_csv("spamham.csv")
print(data_spam)
print(data_spam["text"])
print(data_spam["spam"])

                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5723  Subject: re : research and development charges...     0
5724  Subject: re : receipts from visit  jim ,  than...     0
5725  Subject: re : enron case study update  wow ! a...     0
5726  Subject: re : interest  david ,  please , call...     0
5727  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]
0       Subject: naturally irresistible your corporate...
1       Subject: the stock trading gunslinger  fanny i...
2       Subject: unbelievable new homes made easy  im ...
3       Subject: 4 color printing special  request add...

In [2]:
## Récupérer les instances (X) et les classes (y) et vectoriser
y = data_spam["spam"]

from sklearn.feature_extraction.text import CountVectorizer
V = CountVectorizer(ngram_range = (1,2) )
X = V.fit_transform(data_spam["text"])

## séparer train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
#classifier
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.1, random_state=0)
ppn.fit(X_train, y_train)
y_pred = ppn.predict(X_test)

# On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

Bons résultats 1685
Erreurs: 34


In [4]:
## Réf: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
## à explorer: 
##   ngram_range = (min, max), par défaut (1,1)
##   stop_words
##   lowercase
##   max_features
#--> 1-grams
#--> 2-grams
# ce sont des approches sac de mots (BOW: Bag of Words)
# on ajoutera à notre séance suivante des caractéristiques plus riches

In [5]:
## Avec plusieurs classifieurs
##Comme les choses se répêtent, on va factoriser tout ça :
## On fait tous les imports, on groupe les classifieurs, et on fait une boucle :
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
liste_classifieurs= [
    ["Perceptron", Perceptron(eta0=0.1, random_state=0)],
    ["Logistic Regression", LogisticRegression()],
    ["Decision Tree", DecisionTreeClassifier()],
    ["SVM", SVC(gamma="scale")],
    ["linear_svc", SVC(kernel='linear')],#Plus puissant que le SVC de base mais aussi plus long
#    ["KNN", KNeighborsClassifier()]
]
## Pour éviter les warnings (merci julien):
import warnings 
warnings.filterwarnings("ignore")

In [136]:
## chargement des expériences déjà faites
import json
import os
chemin_expes = "dic_expes_spams.json"
if os.path.exists(chemin_expes):
    f = open(chemin_expes)
    dic_expes = json.load(f)
    f.close()
else:
    dic_expes = {}
print("Expériences stockées : %s"%len(dic_expes))

Expériences stockées : 15


In [137]:
en_minuscules,enlever_stopwords  = False, False

for min_N in range(1, 2):
  for max_N in range(min_N, 2):
    V = CountVectorizer(ngram_range = (min_N, max_N))
    X = V.fit_transform(data_spam["text"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print( "-" * 15)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
        expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules])
        if expe in dic_expes:
            print("  Déjà vu")
            print("  ",expe, score)
            score = dic_expes[expe]
        else:
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = score
            print('  %s classifier : %.4f'%(nom, score))
            pred = clf.predict(X_test)

---------------
Ngram_range : (1, 1)
  Déjà vu
   ['Perceptron', 1, 1, False, False] 0.954043048283886
  Déjà vu
   ['Logistic Regression', 1, 1, False, False] 0.9837114601512508
  Déjà vu
   ['Decision Tree', 1, 1, False, False] 0.9883653286794648
  SVM classifier : 0.9540
  linear_svc classifier : 0.9872
  rbf_svc classifier : 0.9540


BILAN: les 1 grammes ça a l'air d'être le mieux hors autres pré-traitements

In [126]:
from sklearn.feature_extraction import stop_words

min_N, max_N = 1, 1

for enlever_stopwords in [False, True]:
  liste_stopwords = None
  if enlever_stopwords==True:
    liste_stopwords = stop_words.ENGLISH_STOP_WORDS
    
  for en_minuscules in [False, True]:
    print(f"Stopwords {enlever_stopwords}, Minuscules : {en_minuscules}")
    for max_F in [100]:
        V = CountVectorizer(lowercase = en_minuscules, stop_words =  liste_stopwords, max_features = max_F )
        X = V.fit_transform(data_spam["text"])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        for nom, algo in liste_classifieurs:
            clf = algo.fit(X_train, y_train)
            expe = str([nom, min_N, max_N, enlever_stopwords, en_minuscules, max_F])
            if expe in dic_expes:
                print("  Déjà vu :",  expe, score)
                score = dic_expes[expe]
            else:
                print("  Nouvelle expérience")
                print('  %s classifier : %.4f'%(nom, score))
                score = clf.score(X_test, y_test)
                dic_expes[expe] = score

w = open(chemin_expes, "w")
w.write(json.dumps(dic_expes))
w.close()
print("-"*20)
print(f"Expériences stockées dans {chemin_expes}")
print("-"*20)


Stopwords False, Minuscules : False
  Déjà vu : ['Perceptron', 1, 1, False, False, 100] 0.941826643397324
  Déjà vu : ['Logistic Regression', 1, 1, False, False, 100] 0.9435718440954043
  Déjà vu : ['Decision Tree', 1, 1, False, False, 100] 0.9488074461896452
Stopwords False, Minuscules : True
  Déjà vu : ['Perceptron', 1, 1, False, True, 100] 0.9360093077370564
  Déjà vu : ['Logistic Regression', 1, 1, False, True, 100] 0.8900523560209425
  Déjà vu : ['Decision Tree', 1, 1, False, True, 100] 0.9511343804537522
Stopwords True, Minuscules : False
  Déjà vu : ['Perceptron', 1, 1, True, False, 100] 0.9412449098312973
  Déjà vu : ['Logistic Regression', 1, 1, True, False, 100] 0.9488074461896452
  Déjà vu : ['Decision Tree', 1, 1, True, False, 100] 0.9505526468877254
Stopwords True, Minuscules : True
  Déjà vu : ['Perceptron', 1, 1, True, True, 100] 0.9400814426992438
  Déjà vu : ['Logistic Regression', 1, 1, True, True, 100] 0.9383362420011635
  Déjà vu : ['Decision Tree', 1, 1, True, Tru

In [110]:
liste_resultats =[[score, nom_expe] for nom_expe, score in dic_expes.items()]
for res in sorted(liste_resultats):
    print(res)

[0.8900523560209425, ('Perceptron', 1, 1, False, True, 100)]
[0.924956369982548, ('Decision Tree', 2, 2, False, False)]
[0.93717277486911, ('Decision Tree', 1, 1, False, True, 1000)]
[0.9377545084351367, ('Decision Tree', 1, 1, False, False, 100)]
[0.9383362420011635, ('Perceptron', 1, 1, True, True, 100)]
[0.9400814426992438, ('Decision Tree', 1, 1, False, False)]
[0.9412449098312973, ('Decision Tree', 1, 1, True, False, 100)]
[0.9412449098312973, ('Decision Tree', 1, 1, True, True, 100)]
[0.941826643397324, ('Decision Tree', 1, 1, False, True, 100)]
[0.9435718440954043, ('Perceptron', 1, 1, False, False, 100)]
[0.9464805119255381, ('Decision Tree', 1, 1, False, False, 1000)]
[0.9482257126236184, ('Decision Tree', 1, 1, False, True)]
[0.9488074461896452, ('Decision Tree', 1, 1, False, True, 10000)]
[0.9488074461896452, ('Decision Tree', 1, 2, False, False)]
[0.9488074461896452, ('Logistic Regression', 1, 1, False, False, 100)]
[0.9488074461896452, ('Perceptron', 1, 1, True, False, 100

In [112]:
liste_classifieurs= [
    ["Logistic Regression", LogisticRegression()]]
for min_N in range(4, 5):
  for max_N in range(min_N, 7):
    #analyzer: words, char, char_wb
    V = CountVectorizer(ngram_range = (min_N, max_N), analyzer = "char")
    X = V.fit_transform(data_spam["text"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for nom, algo in liste_classifieurs:
            clf = algo.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            dic_expes[expe] = score
            print('%s classifier : %.4f'%(nom, score))


Ngram_range : (4, 4)
Logistic Regression classifier : 0.9889
Ngram_range : (4, 5)
Logistic Regression classifier : 0.9889
Ngram_range : (4, 6)
Logistic Regression classifier : 0.9895
